In [11]:
# example of a cnn for image classification
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import (Conv2D, DepthwiseConv2D,
                                     SeparableConv2D, AveragePooling2D,
                                     Flatten, InputLayer)
from tqdm.notebook import tqdm

In [12]:
from benchmark_utils import run_benchmarks
from parse_utils import parse_benchmark_results
from utils import abs_listdir
def create_conv(out_chan, kernel, **params):
    # TODO: take a look at SeparableConv2D, you haven't used it!
    block = []
    activation = params['activation']
    if params.get('depthwise_conv'):
        if params.get('separable_conv'):
            block += [
                DepthwiseConv2D(kernel_size=(kernel, 1)),
                DepthwiseConv2D(kernel_size=(1, kernel)),
            ]
        else:
            block += [
                DepthwiseConv2D(kernel_size=kernel),
            ]
        block += [
            Conv2D(filters=out_chan, kernel_size=1, activation=activation)
        ]
    else:
        if params.get('separable_conv'):
            block += [ # TODO: How to do this correctly?
                DepthwiseConv2D(kernel_size=(kernel, 1)),
                Conv2D(out_chan, kernel_size=(1, kernel), activation=activation),
            ]
        else:
            block += [
                Conv2D(out_chan, kernel_size=kernel, activation=activation),
            ]
    return block
    

def create_model(in_shape, out_shape, **params):
    model = Sequential([
        InputLayer(input_shape=in_shape),
        *create_conv(out_chan=6, kernel=5, **params),
        AveragePooling2D(pool_size=(2,2)),
        *create_conv(out_chan=16, kernel=5, **params),
        AveragePooling2D(pool_size=(2,2)),
        *create_conv(out_chan=120, kernel=5, **params),
        Flatten(),
        # NOTE: sometimes we can use different activations
        Dense(84, activation=params['activation']),
        Dense(out_shape, activation='softmax')
    ])

    return model

In [13]:
# define model
params_list = []
for depth in [True, False]:
    for sep in [True, False]:
        for act in ['tanh', 'relu', 'elu']:
            params_list.append({'activation':act, 'depthwise_conv': depth, 'separable_conv': sep})

n_classes = 10
in_shape = (32, 32, 3)
models = dict()
for params in params_list:
    name = params['activation'] + ('_depthwise' if params['depthwise_conv'] else '') + ('_spatial' if params['separable_conv'] else '') 
    model = create_model(in_shape, n_classes, **params)
    models[name] = model

In [14]:
# printing model summaries
for name, model in models.items():
    print('-'*80)
    print('NAME ' + name)
    model.summary()

--------------------------------------------------------------------------------
NAME tanh_depthwise_spatial
Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
depthwise_conv2d_36 (Depthwi (None, 28, 32, 3)         18        
_________________________________________________________________
depthwise_conv2d_37 (Depthwi (None, 28, 28, 3)         18        
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 28, 28, 6)         24        
_________________________________________________________________
average_pooling2d_24 (Averag (None, 14, 14, 6)         0         
_________________________________________________________________
depthwise_conv2d_38 (Depthwi (None, 10, 14, 6)         36        
_________________________________________________________________
depthwise_conv2d_39 (Depthwi (None, 10, 10, 6)         36        
__________

In [15]:
# Converting the models to tflite format and saving them
!mkdir -p tflite_models

for name, model in tqdm(models.items()):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()

    # write it to file
    filepath = f'tflite_models/{name}.tflite'
    with open(filepath, 'wb') as f:
        f.write(tflite_model)

**Benchmark tool:** https://github.com/tensorflow/tensorflow/tree/master/tensorflow/lite/tools/benchmark

In [16]:
# create results output directory
!mkdir -p results

# get the filepaths of tflite models from directory
filepaths = abs_listdir('tflite_models/')
run_benchmarks(tf_path="/home/sobir/Downloads/tf", filepaths=filepaths, output_dir='results')

Running: /home/sobir/Downloads/tf/bazel-bin/tensorflow/lite/tools/benchmark/benchmark_model \
--graph=/home/sobir/Desktop/dsproject/tflite_models/relu_depthwise_spatial.tflite \
--enable_op_profiling=true \
--profiling_output_csv_file=/home/sobir/Desktop/dsproject/results/relu_depthwise_spatial_benchmark.txt
STARTING!
Min num runs: [50]
Min runs duration (seconds): [1]
Max runs duration (seconds): [150]
Inter-run delay (seconds): [-1]
Num threads: [1]
Benchmark name: []
Output prefix: []
Min warmup runs: [1]
Min warmup runs duration (seconds): [0.5]
Graph: [/home/sobir/Desktop/dsproject/tflite_models/relu_depthwise_spatial.tflite]
Input layers: []
Input shapes: []
Input value ranges: []
Input layer values files: []
Allow fp16 : [0]
Require full delegation : [0]
Enable op profiling: [1]
Max profiling buffer entries: [1024]
CSV File to export profiling data to: [/home/sobir/Desktop/dsproject/results/relu_depthwise_spatial_benchmark.txt]
Max number of delegated partitions : [0]
Enable pla

In [ ]:
input_files = [f for f in abs_listdir('results/') if f.endswith('.txt')] 
csv_results = parse_benchmark_results(input_files, output_dir='results')

In [ ]:
# display one of the csv files
import pandas as pd
pd.read_csv(csv_results[0])